In [1]:
#getting and working with data
import pandas as pd
import numpy as np
import scipy as sp
import re
import os
from itertools import groupby
import datetime as dt

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('poster', rc={'font.size':35,
                              'axes.titlesize':50,
                              'axes.labelsize':35})

pd.set_option('display.max_rows', 100000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.width', 100000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

### Variable Definitions

- A   = active lever (1 = Left, any other number = Right)
- B   = number of right nose pokes
- C   = number of left nose pokes
#### D   = event time stamp
#### E   = event identity stamp

### Event Stamps

####      46 - Right lever extends 
- trial starts

####      47 - Left lever extends
- trial starts

####      12 - Intertrial interval begins
####      13 - Intertrial interval over and reinforcement available again

####      10 - ITI Right lever press
####      11 - ITI Left lever press
####      14 - ITI Head entry

####      40 - PR reset lever pressed (right)
####      41 - PR reset lever pressed (left)
####      3  - Non-ITI Left lever press
####       4  - Non-ITI Right lever press
####      7  - Non-ITI Head entry
####      17 - Pellet delivery

-      5  - Reinforced left lever press
-      6  - Reinforced right lever press
-      8  - Non-ITI Right nose poke
-      9  - Non-ITI Left nose poke
-      15 - ITI Right nose-poke
-      16 - ITI Left nose-poke
-      18 - Tone delivery
-      42 - Right cue light active
-      43 - Left cue light active
-      44 - Right cue light off
-      45 - Left cue light off
-      48 - Right lever retracts
-      49 - Left lever retracts
-      50 - Head entry to initiate trial
-     100 - End of session marker

In [2]:
def get_file_info_common(file_path_name):
    # takes in a file path, finds the date, animal_number, session, session number, and sub_session, saves each accordingly
    import re
    
    task = file_path_name.split('/')[-1].split('_')[0]
    session = int(file_path_name.split('/')[-1].split('_')[1])
    animal = int(file_path_name.split('/')[-1].split('_')[2])
    date = file_path_name.split('/')[-1].split('_')[3]

    return task, session, animal, date

In [3]:
def make_array_from_path(file_path_name):
    # takes in file path, creates array (each row of array is a row from original med file)
    import numpy as np
    import pandas as pd
    
    data = pd.read_table(file_path_name)
    data_table = pd.DataFrame(data = data)
    data_array = data_table.values
    
    return data_array

In [4]:
def get_events_and_times(array, event_letter, time_letter):
    #takes in array and the letters Med PC program uses to signify an event_stamp and time_stamp; 
    #finds all event_stamps and corresponding time_stamps; creats pandas dataframe
    import re
    import numpy as np
    import pandas as pd

    i = 0
    event_data = []
    time_data = []
    while i < len(array):
        if re.search(event_letter, str(array[i])):
            i = i + 1
            while re.search('\[\'\ \ ', str(array[i])):
                split = array[i][0].split()[1:]
                for element in split:
                    event_data.append(float(element))
                i = i + 1
        elif re.search(time_letter, str(array[i])):
            i = i + 1
            while re.search('\[\'\ \ ', str(array[i])):
                split = array[i][0].split()[1:]
                for element in split:
                    time_data.append(float(element))
                i = i + 1
        else:
            i = i + 1
    
    eandt = np.column_stack((event_data, time_data))
    es_et = pd.DataFrame(data = eandt)
    es_et.columns = ['event_stamp', 'time_stamp']
    
    return es_et

In [5]:
def get_trial_times_LPD(events_and_times, right_extends, left_extends, ITI_start, ITI_end, session_end):
    #takes in dataframe of events and corresponding time_stamps and the Med PC program code for start and end of trial segment
    #e.g. conditioned stimulus onset and offset, ITI onset and offset - here both levers extend so can just use right lever as start
    #finds time_stamps for each start and end, puts into new dataframe
    import numpy as np
    import pandas as pd

    start_time = events_and_times[(events_and_times.event_stamp == right_extends)]['time_stamp'].values
    end_time = events_and_times[(events_and_times.event_stamp == ITI_start) | (events_and_times.event_stamp == session_end)]['time_stamp'].values
    start_time_ITI = events_and_times[events_and_times.event_stamp == ITI_start]['time_stamp'].values
    end_time_ITI = events_and_times[events_and_times.event_stamp == ITI_end]['time_stamp'].values
    
    times = pd.DataFrame(data = [start_time, end_time, start_time_ITI, end_time_ITI])
    times = times.T
    times.columns = ['trial_start', 'trial_end', 'ITI_start', 'ITI_end']
    
    return times

In [7]:
def create_trial_df(events_and_times, trial_times, trial_type):
    #takes in df of events and times and df of trial times
    #creates tidy df containing events and times according to trials
    i = 0
    trial_data = pd.DataFrame()
    
    if trial_type == 'ITI':
        while i < len(trial_times):
            data = events_and_times[(events_and_times['time_stamp'] >= trial_times['ITI_start'][i]) & (events_and_times['time_stamp'] <= trial_times['ITI_end'][i])]
            data['trial_#'] = [i]*len(data)
            data['ITI_start'] = trial_times['ITI_start'][i]
            data['ITI_end'] = trial_times['ITI_end'][i]
            trial_data = pd.concat([trial_data, data], axis = 0, ignore_index=True)
            i = i + 1
    elif trial_type == 'within':
        while i < len(trial_times):
            data = events_and_times[(events_and_times['time_stamp'] >= trial_times['trial_start'][i]) & (events_and_times['time_stamp'] <= trial_times['trial_end'][i])]
            data['trial_#'] = [i]*len(data)
            data['trial_start'] = trial_times['trial_start'][i]
            data['trial_end'] = trial_times['trial_end'][i]
            trial_data = pd.concat([trial_data, data], axis = 0, ignore_index=True)
            i = i + 1
    
    return trial_data

In [8]:
def count(trial_data, event_num):
    #get count of event_num
    trials = len((trial_data['trial_#'].unique()))
    i = 0
    count = []
    while i < trials:
        data = trial_data[trial_data['trial_#'] == i]
        count.append(len(data[data['event_stamp'] == event_num]))
        i = i + 1
    
    return count

In [9]:
def latency(trial_data, event_num, trial_type):
    #get latency of time between trial_start and event_num
    trials = len((trial_data['trial_#'].unique()))
    i = 0
    latency_1st = []
    latency_ave = []
    while i < trials:
        data = trial_data[trial_data['trial_#'] == i]
        data = data[data['event_stamp'] == event_num]
        if len(data) < 1:
            latency_1st.append(np.nan)
            latency_ave.append(np.nan)
        else:
            if trial_type == 'ITI':
                latency_1st.append((data['time_stamp'] - data['ITI_start']).values[0]/100)
                latency_ave.append((data['time_stamp'] - data['ITI_start']).values.mean()/100)
            elif trial_type == 'within':
                latency_1st.append((data['time_stamp'] - data['trial_start']).values[0]/100)
                latency_ave.append((data['time_stamp'] - data['trial_start']).values.mean()/100)
        i = i + 1
    
    return latency_1st, latency_ave

In [10]:
def create_session_df(trial_data, event_dic, trial_type):
    
    trials = len((trial_data['trial_#'].unique()))
    
    #get times
    if trial_type == 'ITI':
        times_df = pd.DataFrame(data = [trial_data.groupby('trial_#')['ITI_start'].unique(), trial_data.groupby('trial_#')['ITI_end'].unique()])
        times_df = times_df.T
    elif trial_type == 'within':
        times_df = pd.DataFrame(data = [trial_data.groupby('trial_#')['trial_start'].unique(), trial_data.groupby('trial_#')['trial_end'].unique()])
        times_df = times_df.T  
        
    i = 0
    trial_times = []

    while i < times_df.shape[0]:
        if trial_type == 'ITI':
            trial_time = (times_df.iloc[i]['ITI_end'] - times_df.iloc[i]['ITI_start'])/100
        elif trial_type == 'within':
            trial_time = (times_df.iloc[i]['trial_end'] - times_df.iloc[i]['trial_start'])/100
        trial_times.append(trial_time[0])
        i += 1
    
    session_data = pd.DataFrame(index=np.arange(0,trials))

    for name, event in event_dic.items():
        session_data[str(name + '_count')] = count(trial_data, event)
        session_data[str(name + '_latency_1st')], session_data[str(name + '_latency_ave')] = latency(trial_data, event, trial_type)
    
    #add bin cout (6 bins of 10 min each)
    if trial_type == 'ITI':
        session_data['trial_bin'] = pd.cut(times_df['ITI_end'].str[0].values/100, bins=6, labels=False)
    elif trial_type == 'within':
        session_data['trial_bin'] = pd.cut(times_df['trial_end'].str[0].values/100, bins=6, labels=False)
    
    session_data['trial_num'] = np.arange(session_data.shape[0])
    session_data['trial_duration'] = trial_times
    
    return session_data

### Read in animal meta data

In [12]:
#read animal info (lever assignment, group etc) into df

path_animal_info = '/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/assignments/2021April_meta.xlsx'
    
animal_info = pd.read_excel(path_animal_info)
animal_info = pd.DataFrame(data = animal_info)

animal_info.head()

,animal,cage,group,lever
0,1215,SA319,4,right
1,1216,SA319,4,right
2,1217,SA319,4,right
3,1236,SA326,5,right
4,1237,SA326,5,right


### Get file paths

In [17]:
path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever'

file_names = os.listdir(path)
print(len(file_names))

file_paths = []

for file in file_names:
    
    file_path = path + '/' + file
    file_paths.append(file_path)

print(len(file_paths))
file_paths[0]

274
274


'/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDFR3R1_1_1230_210331'

### Process data

In [18]:
%%time
final_data = pd.DataFrame()

bit_dic = {'reinforcement': 17, 'right_lever': 4, 'left_lever': 3, 'HE': 7}
ITI_dic = {'reinforcement': 17, 'right_lever': 10, 'left_lever': 11, 'HE': 14}

for file in file_paths:

    print(file) 
    
    if file.split('/')[-1] == '.DS_Store':
        continue
    
    #proccess file
    task, session, animal, date = get_file_info_common(file)
    array = make_array_from_path(file)
    events_and_times = get_events_and_times(array, "E", "D")
    trial_times = get_trial_times_LPD(events_and_times, 46, 47, 12, 13, 100)
    trial_data = create_trial_df(events_and_times, trial_times, 'within')
    ITItrial_data = create_trial_df(events_and_times, trial_times, 'ITI')
    session_data = create_session_df(trial_data, bit_dic, 'within')
    ITIsession_data = create_session_df(ITItrial_data, ITI_dic, 'ITI')
    
    #get animal metadata
    active_lever = animal_info[animal_info['animal'] == int(animal)]['lever'].iloc[0]
    group = int(animal_info[animal_info['animal'] == int(animal)]['group'])
    cage = animal_info[animal_info['animal'] == int(animal)]['cage'].iloc[0]
    
    #create pandas tidy df for within session
    session_data['task'] = [task]*(session_data.shape[0])
    session_data['part'] = ['within']*(session_data.shape[0]) 
    session_data['date'] = [date]*(session_data.shape[0])
    session_data['animal'] = [animal]*(session_data.shape[0])
    session_data['session'] = [session]*(session_data.shape[0])
    session_data['active_lever'] = [active_lever]*(session_data.shape[0])
    session_data['group'] = [group]*(session_data.shape[0])
    session_data['cage'] = [cage]*(session_data.shape[0])
    
    session_data = session_data[['task', 'part', 'date', 'session', 'cage', 'animal', 'group', 'active_lever', 
                                 'trial_num', 'trial_bin', 'trial_duration', 'reinforcement_count', 'reinforcement_latency_1st', 
                                 'HE_count', 'HE_latency_1st', 'HE_latency_ave',
                                 'right_lever_count', 'right_lever_latency_1st',
                                 'left_lever_count', 'left_lever_latency_1st',
                                 'reinforcement_latency_ave', 'right_lever_latency_ave', 'left_lever_latency_ave']]
    
    #create pandas tidy df for within ITI
    ITIsession_data['task'] = [task]*(ITIsession_data.shape[0]) 
    ITIsession_data['part'] = ['ITI']*(ITIsession_data.shape[0]) 
    ITIsession_data['date'] = [date]*(ITIsession_data.shape[0])
    ITIsession_data['animal'] = [animal]*(ITIsession_data.shape[0])
    ITIsession_data['session'] = [session]*(ITIsession_data.shape[0])
    ITIsession_data['active_lever'] = [active_lever]*(ITIsession_data.shape[0])
    ITIsession_data['group'] = [group]*(ITIsession_data.shape[0])
    ITIsession_data['cage'] = [cage]*(ITIsession_data.shape[0])
    
    ITIsession_data = ITIsession_data[['task', 'part', 'date', 'session', 'cage', 'animal', 'group', 'active_lever', 
                                 'trial_num', 'trial_bin', 'trial_duration', 'reinforcement_count', 'reinforcement_latency_1st', 
                                 'HE_count', 'HE_latency_1st', 'HE_latency_ave',
                                 'right_lever_count', 'right_lever_latency_1st',
                                 'left_lever_count', 'left_lever_latency_1st',
                                 'reinforcement_latency_ave', 'right_lever_latency_ave', 'left_lever_latency_ave']]

    #add to final data frame
    data_int = pd.concat([session_data, ITIsession_data], axis = 0, ignore_index=True)
    if final_data.shape[0] == 0:
        final_data = data_int
    else:
        final_data = pd.concat([final_data, data_int], axis = 0, ignore_index=True)

final_data.tail()

/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDFR3R1_1_1230_210331
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDE_3_1219_210408
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDPRR1_1_1230_210401
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_1_1232_210315
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_3_1249_210317
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDPRR1_1_1218_210401
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDE_2_1232_210407
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD

/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_3_1228_210317
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_1_1230_210315
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_2_1248_210316
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDE_2_1230_210407
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDE_4_1243_210409
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_4_1225_210318
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_5_1226_210319
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_7_1250_21032

/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_7_1219_210326
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_8_1227_210405
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDFR3R3_1_1226_210402
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDE_4_1218_210409
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDFR3R3_1_1249_210402
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_8_1248_210405
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_6_1224_210322
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_7_1

/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDPRR3_1_1227_210403
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_7_1228_210326
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDPRR3_1_1244_210403
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_6_1226_210322
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_4_1243_210318
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDFR3R3_1_1242_210402
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_8_1243_210405
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDFR

/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDPRR1_1_1227_210401
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDFR3R1_1_1248_210331
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDPRR1_1_1244_210401
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_2_1243_210316
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDE_1_1250_210406
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD_1_1225_210315
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPDE_2_1225_210407
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round2/orig_lever/LPD

,task,part,date,session,cage,animal,group,active_lever,trial_num,trial_bin,trial_duration,reinforcement_count,reinforcement_latency_1st,HE_count,HE_latency_1st,HE_latency_ave,right_lever_count,right_lever_latency_1st,left_lever_count,left_lever_latency_1st,reinforcement_latency_ave,right_lever_latency_ave,left_lever_latency_ave
24036,LPD,ITI,210317,3,SA324,1231,4,left,49,5,59.99,1,0.0,1,1.15,1.15,0,NaN,0,NaN,0.0,NaN,NaN
24037,LPD,ITI,210317,3,SA324,1231,4,left,50,5,89.99,1,0.0,1,2.21,2.21,0,NaN,1,0.06,0.0,NaN,0.06
24038,LPD,ITI,210317,3,SA324,1231,4,left,51,5,29.99,1,0.0,1,1.04,1.04,0,NaN,0,NaN,0.0,NaN,NaN
24039,LPD,ITI,210317,3,SA324,1231,4,left,52,5,59.99,1,0.0,1,1.15,1.15,0,NaN,1,0.06,0.0,NaN,0.06
24040,LPD,ITI,210317,3,SA324,1231,4,left,53,5,89.99,1,0.0,1,1.02,1.02,0,NaN,0,NaN,0.0,NaN,NaN


In [19]:
#make new df with combined data
df_final = final_data.copy()

lp_train_df_trial = pd.DataFrame()

animals = df_final['animal'].unique()

for animal in animals:
    
    #get animal data
    animal_data = df_final[df_final['animal'] == animal]
    
    #generate session df
    sessions = animal_data['session'].unique()
    
    for session in sessions:
        #get session data for animal
        animal_data_session = animal_data[animal_data['session'] == session]
        
        #compute and fill session data
        if animal_data_session['active_lever'].unique() == 'right':
            animal_data_session['active_lever_count'] = animal_data_session['right_lever_count']
            animal_data_session['active_lever_latency_1st'] = animal_data_session['right_lever_latency_1st']
            animal_data_session['active_lever_latency_ave'] = animal_data_session['right_lever_latency_ave']
            animal_data_session['inactive_lever_count'] = animal_data_session['left_lever_count']
            animal_data_session['inactive_lever_latency_1st'] = animal_data_session['left_lever_latency_1st']
            animal_data_session['inactive_lever_latency_ave'] = animal_data_session['left_lever_latency_ave']

        elif animal_data_session['active_lever'].unique() == 'left':
            animal_data_session['active_lever_count'] = animal_data_session['left_lever_count']
            animal_data_session['active_lever_latency_1st'] = animal_data_session['left_lever_latency_1st']
            animal_data_session['active_lever_latency_ave'] = animal_data_session['left_lever_latency_ave']
            animal_data_session['inactive_lever_count'] = animal_data_session['right_lever_count']
            animal_data_session['inactive_lever_latency_1st'] = animal_data_session['right_lever_latency_1st']
            animal_data_session['inactive_lever_latency_ave'] = animal_data_session['right_lever_latency_ave']
            
        lp_train_df_trial = lp_train_df_trial.append(animal_data_session)
        
lp_train_df_trial.reset_index(inplace=True, drop=True)    

lp_train_df_trial = lp_train_df_trial[['task', 'part', 'date', 'session', 'cage', 'animal', 'group', 'active_lever', 
                                       'trial_num', 'trial_bin', 'trial_duration',
                                       'reinforcement_count', 'reinforcement_latency_1st',
                                       'HE_count', 'HE_latency_1st', 'HE_latency_ave',
                                       'active_lever_count', 'active_lever_latency_1st',
                                       'inactive_lever_count', 'inactive_lever_latency_1st', 
                                       'active_lever_latency_ave', 'inactive_lever_latency_ave', 'reinforcement_latency_ave']]

lp_train_df_trial['lever_ratio'] = lp_train_df_trial['active_lever_count'] / (lp_train_df_trial['active_lever_count'] + lp_train_df_trial['inactive_lever_count'])

print(lp_train_df_trial.shape)
lp_train_df_trial.head()

(24041, 24)


,task,part,date,session,cage,animal,group,active_lever,trial_num,trial_bin,trial_duration,reinforcement_count,reinforcement_latency_1st,HE_count,HE_latency_1st,HE_latency_ave,active_lever_count,active_lever_latency_1st,inactive_lever_count,inactive_lever_latency_1st,active_lever_latency_ave,inactive_lever_latency_ave,reinforcement_latency_ave,lever_ratio
0,LPDFR3R1,within,210331,1,SA324,1230,4,left,0,0,6.73,1,6.73,0,NaN,NaN,3,5.33,0,NaN,6.020000,NaN,6.73,1.0
1,LPDFR3R1,within,210331,1,SA324,1230,4,left,1,0,3.73,1,3.73,0,NaN,NaN,3,2.71,0,NaN,3.203333,NaN,3.73,1.0
2,LPDFR3R1,within,210331,1,SA324,1230,4,left,2,0,4.37,1,4.37,0,NaN,NaN,3,3.03,0,NaN,3.800000,NaN,4.37,1.0
3,LPDFR3R1,within,210331,1,SA324,1230,4,left,3,0,6.18,1,6.18,0,NaN,NaN,3,5.13,0,NaN,5.636667,NaN,6.18,1.0
4,LPDFR3R1,within,210331,1,SA324,1230,4,left,4,0,6.42,1,6.42,1,3.67,3.67,3,2.26,0,NaN,4.853333,NaN,6.42,1.0


In [20]:
lp_train_df_trial.to_csv('LPD.csv', index=False)

In [ ]:
file_path = file_paths[7]
print('file_path:', '\n', file_path, '\n')

task, session, animal, date = get_file_info_common(file_path)
print('date:', date, 'animal:', animal, 'task:', task, 'session:', session, '\n')

array = make_array_from_path(file_path)
print(array[0:50], '\n')

events_and_times = get_events_and_times(array, "E:", "T:")
print(events_and_times.head(), '\n')

trial_times = get_trial_times_LPD(events_and_times, 46,47,12,13,100)
print(trial_times.head(), '\n')

trial_data = create_trial_df(events_and_times, trial_times, 'within')
print(trial_data.head(), '\n')

ITI_trial_data = create_trial_df(events_and_times, trial_times, 'ITI')
print(ITI_trial_data.head(), '\n')

bit_dic = {'reinforcement': 17, 'right_lever': 4, 'left_lever': 3, 'HE_trial': 7}
session_data = create_session_df(trial_data, bit_dic, 'within')

ITI_dic = {'right_lever': 10, 'left_lever': 11, 'HE_trial': 14}
ITI_data = create_session_df(ITI_trial_data, ITI_dic, 'ITI')